# 02 - Creación de Splits (Patient-Level)

**Objetivo**: Dividir el dataset en conjuntos de entrenamiento, validación y prueba garantizando que **no haya fuga de datos (data leakage)**.

**Estrategia**:
- **Nivel**: Paciente (todos los registros de un paciente van al mismo set).
- **Proporción**: Train (60%) / Dev (20%) / Test (20%).
- **Estratificación**: Basada en la clase mayoritaria de cada paciente.

In [4]:
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from collections import Counter

# Importar utilidades compartidas
current_dir = Path.cwd()
if current_dir.name == "notebooks":
    sys.path.append(str(current_dir.parent))
else:
    sys.path.append(str(current_dir))

try:
    from notebooks.utils_shared import setup_paths, validate_file_exists, normalize_label
except ImportError:
    sys.path.append(str(current_dir))
    from utils_shared import setup_paths, validate_file_exists, normalize_label

paths = setup_paths()
DATA_PATH = paths['DATA_PATH']
SPLITS_PATH = paths['SPLITS_PATH']
INPUT_FILE = DATA_PATH / 'ips_clean.csv'

RANDOM_STATE = 42
TEST_SIZE = 0.2
DEV_SIZE = 0.2

print(f"[OK] Paths configurados. Splits se guardarán en: {SPLITS_PATH}")

[OK] Paths configurados. Splits se guardarán en: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/splits


## 1. Cargar Datos Limpios

In [5]:
validate_file_exists(INPUT_FILE, "Ejecuta 01_eda.ipynb primero para generar ips_clean.csv")

df = pd.read_csv(INPUT_FILE)
print(f"[OK] Dataset cargado: {len(df)} registros")
df.head()

[OK] Dataset cargado: 3143 registros


,row_id,patient_id,fecha,etiqueta,texto
0,0,406231.0,16/05/2025,ansiedad,Reposicion de medicacion 2) EXAMEN FISICO GRAL...
1,1,406231.0,14/04/2025,ansiedad,acude para reposicion 2) EXAMEN FISICO GRAL. Y...
2,2,406231.0,19/03/2025,ansiedad,"Se encuentra estable, tranquila, refiere buen ..."
3,3,406231.0,13/03/2025,ansiedad,reposicion segun indicaciones de tratante 2) E...
4,4,406231.0,13/02/2025,ansiedad,reposicion segun indicaciones de tratante 2) E...


## 2. Análisis de Pacientes y Estratificación

In [6]:
PATIENT_COL = 'patient_id' if 'patient_id' in df.columns else 'id_paciente'
LABEL_COL = 'etiqueta'

# --- FIX: Limpiar IDs de pacientes ---
initial_len = len(df)
df = df.dropna(subset=[PATIENT_COL])
dropped = initial_len - len(df)
if dropped > 0:
    print(f"[WARNING] Se eliminaron {dropped} registros con {PATIENT_COL} nulo.")

# Convertir a entero si es float
try:
    df[PATIENT_COL] = df[PATIENT_COL].astype(int)
except:
    pass
# -------------------------------------

def get_majority_label(patient_id, df):
    labels = df[df[PATIENT_COL] == patient_id][LABEL_COL]
    return Counter(labels).most_common(1)[0][0]

patients = df[PATIENT_COL].unique()
patient_labels = {p: get_majority_label(p, df) for p in patients}

patients_df = pd.DataFrame({
    'patient_id': list(patient_labels.keys()),
    'label': list(patient_labels.values())
})

print(f"Pacientes únicos: {len(patients_df)}")
print("Distribución de pacientes por clase mayoritaria:")
print(patients_df['label'].value_counts())

[WARNING] Se eliminaron 12 registros con patient_id nulo.
Pacientes únicos: 89
Distribución de pacientes por clase mayoritaria:
label
depresion    56
ansiedad     33
Name: count, dtype: int64


## 3. Split de Pacientes (Train/Dev/Test)

In [7]:
# 1. Separar Test (20%)
train_dev_patients, test_patients = train_test_split(
    patients_df['patient_id'],
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=patients_df['label']
)

# 2. Separar Train (60%) y Dev (20%) del restante 80%
# Dev es 25% del restante (0.25 * 0.8 = 0.2)
train_patients, dev_patients = train_test_split(
    train_dev_patients,
    test_size=0.25,
    random_state=RANDOM_STATE,
    stratify=patients_df[patients_df['patient_id'].isin(train_dev_patients)]['label']
)

print(f"Pacientes Train: {len(train_patients)} ({len(train_patients)/len(patients):.1%})")
print(f"Pacientes Dev:   {len(dev_patients)} ({len(dev_patients)/len(patients):.1%})")
print(f"Pacientes Test:  {len(test_patients)} ({len(test_patients)/len(patients):.1%})")

# Verificación de leakage
assert len(set(train_patients) & set(dev_patients)) == 0
assert len(set(train_patients) & set(test_patients)) == 0
assert len(set(dev_patients) & set(test_patients)) == 0
print("[OK] No hay pacientes compartidos entre splits.")

Pacientes Train: 53 (59.6%)
Pacientes Dev:   18 (20.2%)
Pacientes Test:  18 (20.2%)
[OK] No hay pacientes compartidos entre splits.


## 4. Generar Archivos de Indices

In [8]:
# Asignar row_id si no existe
if 'row_id' not in df.columns:
    df['row_id'] = df.index

# Filtrar índices
train_idx = df[df[PATIENT_COL].isin(train_patients)]['row_id']
dev_idx = df[df[PATIENT_COL].isin(dev_patients)]['row_id']
test_idx = df[df[PATIENT_COL].isin(test_patients)]['row_id']

print(f"Registros Train: {len(train_idx)}")
print(f"Registros Dev:   {len(dev_idx)}")
print(f"Registros Test:  {len(test_idx)}")

# Guardar dataset base estandarizado
df_base = df[['row_id', PATIENT_COL, 'texto', LABEL_COL]].copy()
df_base.columns = ['row_id', 'patient_id', 'texto', 'etiqueta']
df_base.to_csv(SPLITS_PATH / 'dataset_base.csv', index=False)

# Guardar índices
pd.DataFrame({'row_id': train_idx}).to_csv(SPLITS_PATH / 'train_indices.csv', index=False)
pd.DataFrame({'row_id': dev_idx}).to_csv(SPLITS_PATH / 'dev_indices.csv', index=False)
pd.DataFrame({'row_id': test_idx}).to_csv(SPLITS_PATH / 'test_indices.csv', index=False)

print(f"[OK] Archivos guardados en {SPLITS_PATH}")

Registros Train: 1867
Registros Dev:   627
Registros Test:  637
[OK] Archivos guardados en /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/splits
